In [196]:
from dotenv import load_dotenv
from openai import OpenAI
from elevenlabs.client import AsyncElevenLabs
import asyncio
import json
import os
import argparse
import concurrent.futures

from manga_extraction import extract_all_pages_as_images, save_important_pages, split_volume_into_parts, save_all_pages, extract_panels, scale_base64_image
from vision_analysis import analyze_images_with_gpt4_vision, detect_important_pages, get_important_panels, VISION_PRICE_PER_TOKEN 
from prompts import DRAMATIC_PROMPT, BASIC_PROMPT, BASIC_PROMPT_WITH_CONTEXT,  BASIC_INSTRUCTIONS, KEY_PAGE_IDENTIFICATION_INSTRUCTIONS, KEY_PANEL_IDENTIFICATION_PROMPT, KEY_PANEL_IDENTIFICATION_INSTRUCTIONS
from citation_processing import extract_text_and_citations, extract_script
from movie_director import make_movie
load_dotenv()  # Load environment variables from .env file

True

In [197]:
volume_number = 1
manga = "vinland-saga"

In [198]:
# Initialize OpenAI client with API key
client = OpenAI()
# get elevenlabs api key from dotenv
narration_client = AsyncElevenLabs(api_key=os.getenv("ELEVENLABS_API_KEY"))

print("Extracting all pages from the volume...")
volume_scaled_and_unscaled = extract_all_pages_as_images(f"{manga}/v{volume_number}/v{volume_number}.pdf")
volume = volume_scaled_and_unscaled["scaled"]
volume_unscaled = volume_scaled_and_unscaled["full"]
print("Total pages in volume:", len(volume))

Extracting all pages from the volume...
Total pages in volume: 465


In [199]:
profile_reference = extract_all_pages_as_images(f"{manga}/profile-reference.pdf")["scaled"]
chapter_reference = extract_all_pages_as_images(f"{manga}/chapter-reference.pdf")["scaled"]

profile_pages = []
chapter_pages = [] 

important_page_tokens = 0

batch_size = 20

print("Identifying important pages in the volume...")
# Function to wrap the detect_important_pages call
def process_batch(start_idx, pages):
    response = detect_important_pages(profile_reference, chapter_reference, pages, client,
        KEY_PAGE_IDENTIFICATION_INSTRUCTIONS, KEY_PAGE_IDENTIFICATION_INSTRUCTIONS)
    return start_idx, response

# Using ThreadPoolExecutor to parallelize API calls
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(0, len(volume), batch_size):
        pages = volume[i:i+batch_size]
        futures.append(executor.submit(process_batch, i, pages))

    for future in concurrent.futures.as_completed(futures):
        start_idx, response = future.result()
        end_index = start_idx + batch_size - 1
        print(f"Processing pages {start_idx} to {min(end_index, len(volume)-1)}")
        
        ip = response["parsed_response"]
        print(json.dumps(ip, indent=2))
        for page in ip:
            if page["type"] == "profile":
                profile_pages.append(page["image_index"] + start_idx)
            elif page["type"] == "chapter":
                chapter_pages.append(page["image_index"] + start_idx)

        important_page_tokens += response["total_tokens"]

profile_pages.sort()
chapter_pages.sort()

print("Total tokens to extract profiles and chapters:", important_page_tokens)
print("\n__________\n")
print("Profile pages:", profile_pages)
print("Chapter pages:", chapter_pages)

Identifying important pages in the volume...
Using GPT as a backup to format JSON object...
Processing pages 0 to 19
[
  {
    "image_index": 2,
    "type": "chapter"
  },
  {
    "image_index": 4,
    "type": "profile"
  }
]
Using GPT as a backup to format JSON object...
Processing pages 180 to 199
[
  {
    "image_index": 14,
    "type": "chapter"
  }
]
Using GPT as a backup to format JSON object...
Processing pages 160 to 179
[
  {
    "image_index": 7,
    "type": "chapter"
  },
  {
    "image_index": 8,
    "type": "chapter"
  }
]
Using GPT as a backup to format JSON object...
Using GPT as a backup to format JSON object...
Processing pages 40 to 59
[]
Processing pages 200 to 219
[
  {
    "image_index": 0,
    "type": "chapter"
  },
  {
    "image_index": 18,
    "type": "chapter"
  },
  {
    "image_index": 19,
    "type": "profile"
  }
]
Using GPT as a backup to format JSON object...
Processing pages 220 to 239
[
  {
    "image_index": 0,
    "type": "chapter"
  }
]
Using GPT as

In [200]:
print(f"{len(volume)}")
print("\n__________\n")
print("Saving important pages to disk for QA...")
save_important_pages(volume, profile_pages, chapter_pages, manga, volume_number)

465

__________

Saving important pages to disk for QA...


In [201]:
character_profiles = [volume[i] for i in profile_pages]    
jobs = split_volume_into_parts(volume, volume_unscaled, chapter_pages, 9)
parts = jobs["parts"]
jobs_unscaled = jobs["unscaled_images"]
jobs = jobs["scaled_images"]

# Summarize the images in the first job
response = analyze_images_with_gpt4_vision(character_profiles, jobs[0], client, BASIC_PROMPT, BASIC_INSTRUCTIONS)
recap = response.choices[0].message.content
tokens = response.usage.total_tokens
movie_script = extract_text_and_citations(response.choices[0].message.content, jobs[0], jobs_unscaled[0])

print("\n\n\n_____________\n\n\n")
print(response.choices[0].message.content)

# iterate thrugh the rest of the jobs while adding context from previous ones
for i, job in enumerate(jobs):
    if i == 0:
        continue
    response = analyze_images_with_gpt4_vision(character_profiles, job, client, recap + "\n-----\n" + BASIC_PROMPT_WITH_CONTEXT, BASIC_INSTRUCTIONS)
    recap = recap + "\n\n" + response.choices[0].message.content
    tokens += response.usage.total_tokens
    print("\n\n\n_____________\n\n\n")
    print(response.choices[0].message.content)
    movie_script = movie_script + extract_text_and_citations(response.choices[0].message.content, job, jobs_unscaled[i])

print("\n\n\n_____________\n\n\n")
print("\n\n\n_____________\n\n\n")
print("\n\n\n_____________\n\n\n")

narration_script = extract_script(movie_script)
print(narration_script)
print("\n___________\n")

2->89
90->132
133->194
195->243
244->304
305->363
364->419
420->463
464->end (465)



_____________



The scene unfolds amidst the cacophony of war, with fervent cries and the metallic clash of weapons echoing through the Frankish kingdom's territory. At the heart of the chaos, the young and relentless Thorfinn stands determined, harboring a fierce glint in his eyes and the burden of vengeful purpose against Askeladd, his commander[^6][^7][^8]. The battle rages on, with each side maneuvering for the upper hand. Amidst the fray, a plan unfurls as riders are dispatched to face the oncoming French army[^14][^15].

With the snowfall heralding an inevitable siege, the villagers behind the fray carry dread in their hearts, while the warriors, including Thorfinn, station themselves strategically, surveying the opposing forces with a tactical eye[^12][^13]. Confident and mocking, Askeladd commands Thorfinn, who remains stoic and silently focused on his own pursuits of revenge[^16][^17].

The 

In [202]:
extract_panels(movie_script)

Load text detector ... Done!
Number of unscaled images in segment: 1
Loading images ... Number of images loaded: 1


Processing images: 1it [00:00, 16.21it/s]


Number of panels generated: 1
Panels generated for text: The scene unfolds amidst the cacophony of war, with fervent cries and the metallic clash of weapons echoing through the Frankish kingdom's territory. At the heart of the chaos, the young and relentless Thorfinn stands determined, harboring a fierce glint in his eyes and the burden of vengeful purpose against Askeladd, his commander.
Number of unscaled images in segment: 3
Loading images ... Number of images loaded: 3


Processing images: 3it [00:00, 18.60it/s]


Number of panels generated: 3
Panels generated for text: The battle rages on, with each side maneuvering for the upper hand. Amidst the fray, a plan unfurls as riders are dispatched to face the oncoming French army.
Number of unscaled images in segment: 2
Loading images ... Number of images loaded: 2


...

Number of panels generated: 2
Panels generated for text: As the Viking ships sail away, carrying Thors's body and his comrades, Thorfinn, alone and adrift, is consumed by an incendiary rage that promises vengeance: "I swear I'll kill you. I'll kill you!!"^28]. His father, the man of no enemies, leaves a legacy marred by unresolved anguish, setting Thorfinn on a dark path fueled by retribution.
Number of unscaled images in segment: 1
Loading images ... Number of images loaded: 1


Processing images: 1it [00:00, 21.83it/s]


Number of panels generated: 1
Panels generated for text: In the poignant aftermath, the night envelopes all, and the vast sea, indifferent to mortal woes, quietly absorbs the echoes of the fallen warrior and the birth cries of a vengeful shadow.
Number of unscaled images in segment: 0
Loading images ... Number of images loaded: 0


Processing images: 0it [00:00, ?it/s]

Number of panels generated: 0
Panels generated for text: Amidst the bleak expanse of the sea and the merciless winds, Thorfinn's journey with the war band of Askeladd unfolds on a somber map scattered with bloodshed and battles. His tireless pursuit, from his native Iceland to the shores of England, paints a bitter trail of vengeance stretching far across the frigid North Atlantic. Every mark on the map, from Thorfinn's village to the Frankish lands, echoes the relentless cadence of his grim sojourn—each location a stark reminder of violence endured and the will to persist.

The page reveals not just the vast distances traversed but the transformation within Thorfinn, etching the scars of countless skirmishes into his very soul. As the map spans the pivotal points of his travels, one can almost hear the clash of steel and the cries of warriors in resonance with each name etched upon the parchment. An inconspicuous 'X' marks the village of Thorfinn, an origin now synonymous with both lo

In [203]:
print("number of segments:", len(movie_script))
for i, segment in enumerate(movie_script):
    print("segment", i, ": ", segment["text"])
    all_panels_base64 = [panel for sublist in segment["panels"].values() for panel in sublist]
    print(len(all_panels_base64))
    print("number of panels:", len(all_panels_base64))
    print("number of images:", len(segment["images"]))

def process_segment(segment_tuple):
    i, segment = segment_tuple  # Unpack the tuple
    panels = []
    for j, page in enumerate(segment["images"]):
        if "panels" in segment:
            if j not in segment["panels"]:
                panels.append(page)
            else:
                for panel in segment["panels"][j]:
                    panels.append(panel)
        else:
            panels.append(page)
    
    scaled_panels = [scale_base64_image(p) for p in panels]


    response = get_important_panels(profile_reference, scaled_panels, client, 
        segment["text"] + "\n________\n" + KEY_PANEL_IDENTIFICATION_PROMPT, KEY_PANEL_IDENTIFICATION_INSTRUCTIONS)

    important_panels = response["parsed_response"]
    # check if important panels is an array
    if not isinstance(important_panels, list):
        important_panels = []

    ip = []
    for p in important_panels:
        number = p
        if isinstance(number, str):
            if number.isdigit():
                number = int(number)
        if not isinstance(number, int):
            continue

        if number < len(panels):
            ip.append(panels[number])
        
    
    return i, ip, response["total_tokens"]

# Initialize variables
panel_tokens = 0
important_panels_info = {}

# Use ThreadPoolExecutor to parallelize the processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Create a list of futures
    futures = [executor.submit(process_segment, (i, segment)) for i, segment in enumerate(movie_script)]
    
    # Collect the results as they complete
    for future in concurrent.futures.as_completed(futures):
        i, ip, tokens = future.result()
        if ip:
            print("Important panels for segment", i, "exist.")
        else: 
            print("No important panels for segment", i)
        movie_script[i]["important_panels"] = ip  # Assign the important panels back to the segment
        panel_tokens += tokens


ELEVENLABS_PRICE_PER_CHARACTER = 0.0003
print("Tokens for extracting profiles and chapters:", important_page_tokens, " | ", "${:,.4f}".format(VISION_PRICE_PER_TOKEN * important_page_tokens))
print("Tokens for summarization:", tokens,  " | ", "${:,.4f}".format(VISION_PRICE_PER_TOKEN * tokens))
print("Tokens for extracting important panels:", panel_tokens, " | ", "${:,.4f}".format(VISION_PRICE_PER_TOKEN * panel_tokens))
total_gpt_tokens = important_page_tokens + tokens + panel_tokens
print("Total GPT tokens:", total_gpt_tokens,  " | ", "${:,.4f}".format(VISION_PRICE_PER_TOKEN * (total_gpt_tokens)))
print("Total elevenlabs characters:", len(narration_script), " | ", "${:,.4f}".format(ELEVENLABS_PRICE_PER_CHARACTER * (len(narration_script))))
print("GRAND TOTAL COST"," | ", "${:,.4f}".format(VISION_PRICE_PER_TOKEN * (total_gpt_tokens) + ELEVENLABS_PRICE_PER_CHARACTER * (len(narration_script))))





number of segments: 101
segment 0 :  The scene unfolds amidst the cacophony of war, with fervent cries and the metallic clash of weapons echoing through the Frankish kingdom's territory. At the heart of the chaos, the young and relentless Thorfinn stands determined, harboring a fierce glint in his eyes and the burden of vengeful purpose against Askeladd, his commander.
3
number of panels: 3
number of images: 1
segment 1 :  The battle rages on, with each side maneuvering for the upper hand. Amidst the fray, a plan unfurls as riders are dispatched to face the oncoming French army.
9
number of panels: 9
number of images: 3
segment 2 :  With the snowfall heralding an inevitable siege, the villagers behind the fray carry dread in their hearts, while the warriors, including Thorfinn, station themselves strategically, surveying the opposing forces with a tactical eye.
14
number of panels: 14
number of images: 2
segment 3 :  Confident and mocking, Askeladd commands Thorfinn, who remains stoic 

In [204]:
await make_movie(movie_script, manga, volume_number, narration_client)

Narrating movie script...
got bytes: 114102
got bytes: 149211
got bytes: 237818
got bytes: 254537
got bytes: 111177
got bytes: 161750
got bytes: 312633
got bytes: 224862
got bytes: 223608
got bytes: 260806
got bytes: 299258
got bytes: 310543
got bytes: 168437
got bytes: 183484
got bytes: 258298
got bytes: 140434
got bytes: 181812
got bytes: 81502
got bytes: 102400
got bytes: 199784
got bytes: 85263
got bytes: 137508
got bytes: 186409
got bytes: 127477
got bytes: 187245
got bytes: 303020
got bytes: 205635
got bytes: 86099
got bytes: 137926
got bytes: 282122
got bytes: 132911
got bytes: 168437
got bytes: 140852
got bytes: 147121
got bytes: 192261
got bytes: 78576
got bytes: 184320
got bytes: 234475
got bytes: 196022
got bytes: 131239
got bytes: 272509
got bytes: 341472
got bytes: 77740
got bytes: 140852
got bytes: 379506
got bytes: 423392
got bytes: 211069
got bytes: 184737
got bytes: 189335
got bytes: 328097
got bytes: 168437
got bytes: 103235
got bytes: 178886
got bytes: 181812
got byt

MoviePy - Done.
Moviepy - Writing video vinland-saga/v1/recap.mp4



Moviepy - Done !
Moviepy - video ready vinland-saga/v1/recap.mp4
Movie created successfully!


True